In [1]:
import pandas as pd 
import os
import glob
import re
import pickle
import numpy as np
from nltk.tokenize import word_tokenize

In [2]:
# data load
earnings = pd.read_pickle('D:\\Earnings\\data\\earnings_text.pkl')
earnings.head(2)


,ddate,text
0,2023-09-19 18:40:00,"Red Cat Holdings, Inc. (NASDAQ:RCAT) Q1 2024 E..."
1,2023-09-19 18:15:00,Endava plc (NYSE:DAVA) Q4 2023 Earnings Call T...


In [4]:
with open('../data/stop_words.pkl', mode='rb') as pfile:
    stop_words = pickle.load(pfile)

In [8]:
with open('../data/name_dataset/us_names.pkl', mode='rb') as f:
    us_names = pickle.load(f)

with open('../data/name_dataset/gb_names.pkl', mode='rb') as f:
    gb_names = pickle.load(f)

In [34]:
stop_from_data = pd.read_pickle('../data/stoplist_from_data.pkl')

In [49]:
import re
count = 0
def data_clean(text):
    global count
    text = ' '.join([ line.strip().lower() for line in text.split('\n') ])
    number_removed = re.sub('[^a-zA-Z\s]', ' ', text)
    gt_3_letters = [word for word in word_tokenize(number_removed) if len(word) >= 3 ]
    no_stops = [word for word in gt_3_letters if word not in stop_words]
    no_us_names = [word for word in no_stops if word not in us_names.keys()]
    no_gb_names = [word for word in no_us_names if word not in gb_names.keys()]
    no_stop2 = [word for word in no_gb_names if word not in stop_from_data]

    if count%10==0: print(f'printing row: {count}')
    count +=1
    
    return no_stop2



In [52]:
earnings['clean'] = earnings['text'].apply(data_clean)

printing row: 5
printing row: 6
printing row: 7
printing row: 8
printing row: 9
printing row: 10
printing row: 11
printing row: 12
printing row: 13
printing row: 14
printing row: 15
printing row: 16
printing row: 17
printing row: 18
printing row: 19
printing row: 20
printing row: 21
printing row: 22
printing row: 23
printing row: 24
printing row: 25
printing row: 26
printing row: 27
printing row: 28
printing row: 29
printing row: 30
printing row: 31
printing row: 32
printing row: 33
printing row: 34
printing row: 35
printing row: 36
printing row: 37
printing row: 38
printing row: 39
printing row: 40
printing row: 41
printing row: 42
printing row: 43
printing row: 44
printing row: 45
printing row: 46
printing row: 47
printing row: 48
printing row: 49
printing row: 50
printing row: 51
printing row: 52
printing row: 53
printing row: 54
printing row: 55
printing row: 56
printing row: 57
printing row: 58
printing row: 59
printing row: 60
printing row: 61
printing row: 62
printing row: 63
pr

## Topics Modeling

In [118]:
import gensim
from gensim import models, corpora

# load
earnings = pd.read_pickle('D:\\Earnings\\data\\earnings_text_clean.pkl')


In [122]:
earnings_dict = corpora.Dictionary(earnings['clean'].to_list())

In [123]:
earnings_corpus = [earnings_dict.doc2bow(text) for text in earnings['clean']]

In [124]:
NUM_TOPICS = 10
earnings_model = models.LdaModel(earnings_corpus, num_topics=NUM_TOPICS,
                                 id2word=earnings_dict, passes=15) 

In [179]:
for idx, topic in earnings_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n\n')

Topic: 0 
Words: 0.015*"cancer" + 0.010*"program" + 0.010*"disease" + 0.009*"studies" + 0.008*"therapy" + 0.007*"programs" + 0.007*"vaccine" + 0.006*"financial" + 0.006*"efficacy" + 0.006*"research"


Topic: 1 
Words: 0.041*"customers" + 0.022*"customer" + 0.020*"market" + 0.015*"revenue" + 0.014*"product" + 0.012*"enterprise" + 0.010*"opportunity" + 0.010*"network" + 0.010*"technology" + 0.010*"products"


Topic: 2 
Words: 0.025*"market" + 0.013*"rates" + 0.012*"financial" + 0.012*"credit" + 0.012*"investment" + 0.010*"assets" + 0.009*"increased" + 0.009*"management" + 0.009*"performance" + 0.007*"environment"


Topic: 3 
Words: 0.017*"market" + 0.014*"projects" + 0.012*"project" + 0.010*"prices" + 0.009*"financial" + 0.009*"ebitda" + 0.008*"costs" + 0.008*"customers" + 0.008*"opportunities" + 0.008*"assets"


Topic: 4 
Words: 0.017*"revenue" + 0.013*"market" + 0.012*"product" + 0.011*"consumer" + 0.011*"marketing" + 0.011*"ebitda" + 0.010*"adjusted" + 0.010*"performance" + 0.009*"ope

In [183]:
import pickle

# Specify the file path
topics_file_path = 'D:\\Earnings\\data\\topics_10.txt'

# Open the pickle file in binary append mode and save the topics
with open(topics_file_path, mode = 'w', encoding='utf-8') as tfile:
    for idx, topic in earnings_model.print_topics(-1):
        tfile.write(f'Topic: {idx} \nWords: {topic}\n\n')


In [184]:
import pickle

# Specify the file path
topics_file_path = 'D:\\Earnings\\data\\topics_10.txt'

# To load the topics back from the pickle file:
with open(topics_file_path, mode='r', encoding='utf-8') as tfile:
    lines = tfile.readlines()
    for line in lines:
        print(line)

Topic: 0 

Words: 0.015*"cancer" + 0.010*"program" + 0.010*"disease" + 0.009*"studies" + 0.008*"therapy" + 0.007*"programs" + 0.007*"vaccine" + 0.006*"financial" + 0.006*"efficacy" + 0.006*"research"



Topic: 1 

Words: 0.041*"customers" + 0.022*"customer" + 0.020*"market" + 0.015*"revenue" + 0.014*"product" + 0.012*"enterprise" + 0.010*"opportunity" + 0.010*"network" + 0.010*"technology" + 0.010*"products"



Topic: 2 

Words: 0.025*"market" + 0.013*"rates" + 0.012*"financial" + 0.012*"credit" + 0.012*"investment" + 0.010*"assets" + 0.009*"increased" + 0.009*"management" + 0.009*"performance" + 0.007*"environment"



Topic: 3 

Words: 0.017*"market" + 0.014*"projects" + 0.012*"project" + 0.010*"prices" + 0.009*"financial" + 0.009*"ebitda" + 0.008*"costs" + 0.008*"customers" + 0.008*"opportunities" + 0.008*"assets"



Topic: 4 

Words: 0.017*"revenue" + 0.013*"market" + 0.012*"product" + 0.011*"consumer" + 0.011*"marketing" + 0.011*"ebitda" + 0.010*"adjusted" + 0.010*"performance" + 0

In [129]:

# Extract the topic distributions for each document
doc_topics = [earnings_model.get_document_topics(bow, minimum_probability=0.0) for bow in earnings_corpus]

# Convert the topic distributions into a DataFrame
topics_df = pd.DataFrame([{t[0]: t[1] for t in topics} for topics in doc_topics])

# Rename columns for clarity
topics_df.columns = [f"topic_{col}" for col in topics_df.columns]

In [185]:
#pd.concat([earnings, topics_df], axis=1).to_pickle('D:\\Earnings\\data\\earnings_topics.pkl')

In [186]:
#earnings_dict.save('D:\\Earnings\\data\\earnings_dict')
#earnings_model.save('D:\\Earnings\\data\\earnings_model.gensim')

## VADER Sentiment

In [189]:
from vaderSentiment import vaderSentiment

vader_analyzer = vaderSentiment.SentimentIntensityAnalyzer()

In [190]:
earning_clean = pd.read_pickle('D:\\Earnings\\data\earnings_text_clean.pkl')

In [192]:
def apply_vader_sentiment(dataframe: pd.DataFrame=None):
    output_dict = {'neg':[],'neu':[],'pos':[]}
    for idx, row in enumerate(dataframe.to_dict('records')):
        if idx%5==0:
            print(f'working on the: {idx}')
        text = [ line.strip() for line in row['clean']]  # stript escape line character s
        single_sentence = ' '.join(text)
        sentiment_scores = vader_analyzer.polarity_scores(single_sentence)
        output_dict['neg'].append(sentiment_scores['neg'])
        output_dict['neu'].append(sentiment_scores['neu'])
        output_dict['pos'].append(sentiment_scores['pos'])
     
    return pd.DataFrame(output_dict)

In [194]:
earnings_vader_df = apply_vader_sentiment(dataframe=earnings)

working on the: 0
working on the: 5
working on the: 10
working on the: 15
working on the: 20
working on the: 25
working on the: 30
working on the: 35
working on the: 40
working on the: 45
working on the: 50
working on the: 55
working on the: 60
working on the: 65
working on the: 70
working on the: 75
working on the: 80
working on the: 85
working on the: 90
working on the: 95
working on the: 100
working on the: 105
working on the: 110
working on the: 115
working on the: 120
working on the: 125
working on the: 130
working on the: 135
working on the: 140
working on the: 145
working on the: 150
working on the: 155
working on the: 160
working on the: 165
working on the: 170
working on the: 175
working on the: 180
working on the: 185
working on the: 190
working on the: 195
working on the: 200
working on the: 205
working on the: 210
working on the: 215
working on the: 220
working on the: 225
working on the: 230
working on the: 235
working on the: 240
working on the: 245
working on the: 250
wo

In [196]:
earnings_vader_df

## Loughran-McDonald Sentiment Analysis

In [197]:
def lm_sentiment_score(text: str=None)-> dict:

    # intialize dictionary
    sentiment_dict = {'LM_Positive':[], 'LM_Negative':[], 'LM_Uncertainty':[]}
    
    # Initialize sentiment scores
    pos_score = 0
    neg_score = 0
    uncertain_score = 0
    
    # upload Loughran-McDonald sentiment dataset
    lm_dataframe = pd.read_csv('../data/LoughranMcDonald/LM-SA-2020.csv') 

    for token in text:
        if token in lm_dataframe['word'].to_list():
            token_row = lm_dataframe[lm_dataframe['word'] == token]
            lm_sent = token_row.iloc[0]['sentiment']
            if lm_sent == 'Positive':
                pos_score+=1
            elif lm_sent == 'Negative':
                neg_score+=1
            elif lm_sent == 'Uncertainty':
                uncertain_score+=1
        else:
            pass
    sentiment_dict['LM_Positive'].append(pos_score)
    sentiment_dict['LM_Negative'].append(neg_score)
    sentiment_dict['LM_Uncertainty'].append(uncertain_score)

    return sentiment_dict


def compute_lm_sentiment(dataframe: pd.DataFrame=None)-> pd.DataFrame:
    results = []
    for ind, row in enumerate(dataframe.to_dict('records')):
        if ind%10==0:
            print(f'printing: {ind}')
        text = row['clean']
        my_sent = lm_sentiment_score(text=text)
        df = pd.DataFrame(my_sent)
        
        results.append(df)
    
    results_concat = pd.concat(results, axis=0)
    results_concat = results_concat.reset_index(drop=True)

    return results_concat

In [199]:
from nltk.tokenize import word_tokenize

results = []
for ind, row in enumerate(earnings.to_dict('records')):
    if ind%10==0:
        print(f'printing: {ind}')
    text = row['clean']
    #text = word_tokenize(text)
    my_sent = lm_sentiment_score(text=text)
    df = pd.DataFrame(my_sent)
        
    results.append(df)
    
results_concat = pd.concat(results, axis=0)
results_concat = results_concat.reset_index(drop=True)
print(results_concat)

printing: 0
printing: 10
printing: 20
printing: 30
printing: 40
printing: 50
printing: 60
printing: 70
printing: 80
printing: 90
printing: 100
printing: 110
printing: 120
printing: 130
printing: 140
printing: 150
printing: 160
printing: 170
printing: 180
printing: 190
printing: 200
printing: 210
printing: 220
printing: 230
printing: 240
printing: 250
printing: 260
printing: 270
printing: 280
printing: 290
printing: 300
printing: 310
printing: 320
printing: 330
printing: 340
printing: 350
printing: 360
printing: 370
printing: 380
printing: 390
printing: 400
printing: 410
printing: 420
printing: 430
printing: 440
printing: 450
printing: 460
printing: 470
printing: 480
printing: 490
printing: 500
printing: 510
printing: 520
printing: 530
printing: 540
printing: 550
printing: 560
printing: 570
printing: 580
printing: 590
printing: 600
printing: 610
printing: 620
printing: 630
printing: 640
printing: 650
printing: 660
printing: 670
printing: 680
printing: 690
printing: 700
printing: 710
pri

In [201]:
#results_concat.to_pickle('D:\\Earnings\\data\\earnings_lm_sentiment.pkl')

# Information  

In [3]:
earnings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ddate   4000 non-null   object
 1   text    4000 non-null   object
dtypes: object(2)
memory usage: 62.6+ KB


In [15]:
earnings = pd.read_pickle('D:\\Earnings\\data\\earnings.pkl')

In [16]:
from datetime import date, datetime
def string_2_datetime(ognoo: date=None):
    if isinstance(ognoo, str):
        return datetime.strptime(ognoo, '%Y-%m-%d')
    else:
        return ognoo


In [17]:
earnings.ddate = earnings.ddate.apply(string_2_datetime)

In [19]:
#earnings.to_pickle('../data/earnings.pkl')

In [1]:
import pandas as pd
aggregate = pd.read_pickle('../data/Aggregated.pkl')

In [25]:
earnings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   url     4000 non-null   object        
 1   tick    4000 non-null   object        
 2   ognoo   4000 non-null   object        
 3   ddate   4000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 125.1+ KB


In [4]:
exchange = pd.read_pickle('D:\\Earnings\\data\\earnings_exchange2.pkl')

In [5]:
exchange.columns = ['id', 'close', 'volume', 'sma50_close','sma200_close']

In [6]:
lm_sent = pd.read_pickle('D:\\Earnings\\data\\earnings_lm_sentiment.pkl')

In [7]:
vader_sma = pd.read_pickle('D:\\Earnings\\data\\earnings_vader.pkl')


In [57]:
#topics10.iloc[:,3:]
#earnings
unseen = pd.concat([exchange,topics10.iloc[:,3:],lm_sent, vader_sma],axis=1)


In [70]:
unseen = pd.read_pickle('D:\\Earnings\\data\\earnings_topics.pkl')

In [78]:
pd.set_option('display.max_columns',None)
vers1 = pd.concat([earnings, exchange, unseen,lm_sent,vader_sma],axis=1)[['id','url', 'tick', 'ognoo', 'ddate', 'close', 'volume', 'sma50_close','sma200_close', 'ddate', 'transcript', 'clean', 'Topic_1', 'Topic_2','Topic_3', 'Topic_4', 'Topic_5', 'Topic_6', 'Topic_7', 'Topic_8',
       'Topic_9', 'Topic_10', 'LM_Positive', 'LM_Negative', 'LM_Uncertainty',
       'neg', 'neu', 'pos']]

In [83]:
vers2 = vers1[['id', 'url', 'tick', 'ognoo', 'close', 'volume',
       'sma50_close', 'sma200_close', 'ddate', 'transcript', 'clean',
       'Topic_1', 'Topic_2', 'Topic_3', 'Topic_4', 'Topic_5', 'Topic_6',
       'Topic_7', 'Topic_8', 'Topic_9', 'Topic_10', 'LM_Positive',
       'LM_Negative', 'LM_Uncertainty', 'neg', 'neu', 'pos']]

In [119]:
vers2.columns = ['id', 'url', 'tick', 'ognoo', 'close', 'volume', 'sma50_close',
       'sma200_close', 'ddate1', 'ddate2', 'ddate3', 'ddate4', 'transcript',
       'clean', 'Topic_1', 'Topic_2', 'Topic_3', 'Topic_4', 'Topic_5',
       'Topic_6', 'Topic_7', 'Topic_8', 'Topic_9', 'Topic_10', 'LM_Positive',
       'LM_Negative', 'LM_Uncertainty', 'neg', 'neu', 'pos']

In [123]:
unseen = vers2[['id', 'url','transcript','clean','ddate1','tick','close', 'volume', 'sma50_close',
       'sma200_close','Topic_1', 'Topic_2', 'Topic_3', 'Topic_4', 'Topic_5',
       'Topic_6', 'Topic_7', 'Topic_8', 'Topic_9', 'Topic_10', 'LM_Positive',
       'LM_Negative', 'LM_Uncertainty', 'neg', 'neu', 'pos']]

In [2]:
aggregate = pd.read_pickle('../data/Aggregated.pkl')

In [15]:
aggregate_short = aggregate[['id','url','date','stock','tick','close','sma50_close','sma200_close']]

In [17]:
agre_topics = pd.read_pickle('../data/large_topics_proportions_df.pkl')

In [19]:
lm = pd.read_pickle('../data/LoughranMcDonald/earnings_sentiment.pkl')

In [21]:
vader = pd.read_pickle('../data/SentimentVader/vader_sentiment.pkl')

In [24]:
aggregate_short = aggregate_short.reset_index(drop=True, inplace=False)


In [42]:
pd.read_pickle('../data/earnings_unseen.pkl')[['id', 'url', 'ddate1', 'tick', 'close', 'sma50_close', 'sma200_close', 'Topic_1', 'Topic_2', 'Topic_3',
       'Topic_4', 'Topic_5', 'Topic_6', 'Topic_7', 'Topic_8', 'Topic_9',
       'Topic_10', 'LM_Positive', 'LM_Negative', 'LM_Uncertainty', 'neg',
       'neu', 'pos']].iloc[:10,:].to_csv('../data/unseen_short.csv', index=False)

In [19]:
#pd.read_pickle('D:\\Earnings\data\\earnings_topics.pkl')
with open('D:\\Earnings\data\\topics_10.txt', mode='r', encoding='utf-8') as tfile:
    reader = tfile.readlines()
    for line in reader:
        print(line)

Topic: 0 

Words: 0.015*"cancer" + 0.010*"program" + 0.010*"disease" + 0.009*"studies" + 0.008*"therapy" + 0.007*"programs" + 0.007*"vaccine" + 0.006*"financial" + 0.006*"efficacy" + 0.006*"research"



Topic: 1 

Words: 0.041*"customers" + 0.022*"customer" + 0.020*"market" + 0.015*"revenue" + 0.014*"product" + 0.012*"enterprise" + 0.010*"opportunity" + 0.010*"network" + 0.010*"technology" + 0.010*"products"



Topic: 2 

Words: 0.025*"market" + 0.013*"rates" + 0.012*"financial" + 0.012*"credit" + 0.012*"investment" + 0.010*"assets" + 0.009*"increased" + 0.009*"management" + 0.009*"performance" + 0.007*"environment"



Topic: 3 

Words: 0.017*"market" + 0.014*"projects" + 0.012*"project" + 0.010*"prices" + 0.009*"financial" + 0.009*"ebitda" + 0.008*"costs" + 0.008*"customers" + 0.008*"opportunities" + 0.008*"assets"



Topic: 4 

Words: 0.017*"revenue" + 0.013*"market" + 0.012*"product" + 0.011*"consumer" + 0.011*"marketing" + 0.011*"ebitda" + 0.010*"adjusted" + 0.010*"performance" + 0

,id,url,transcript,clean,ddate1,tick,close,volume
0,0.0,https://seekingalpha.com/article/4636135-red-c...,"Red Cat Holdings, Inc. (NASDAQ:RCAT) Q1 2024 E...","[rcat, delahoussaye, investor, coreir, hernon,...",2023-09-19 18:40:00,RCAT,1.010000,269000.0
1,1.0,https://seekingalpha.com/article/4636134-endav...,Endava plc (NYSE:DAVA) Q4 2023 Earnings Call T...,"[endava, cotterell, faucette, deutsche, wedbus...",2023-09-19 18:15:00,DAVA,56.540001,1492700.0
2,2.0,https://seekingalpha.com/article/4636133-inves...,"Investcorp Credit Management BDC, Inc. (NASDAQ...","[investcorp, credit, management, icmb, corpora...",2023-09-19 17:43:00,ICMB,3.900000,83400.0
3,3.0,https://seekingalpha.com/article/4636128-naked...,Naked Wines plc (OTCQX:NWINF) Q4 2023 Earnings...,"[naked, otcqx, nwinf, liberum, investec, naked...",2023-09-19 17:19:00,NWINF,0.756000,2400.0
4,4.0,https://seekingalpha.com/article/4636122-seric...,Serica Energy plc (OTCPK:SQZZF) Q2 2023 Earnin...,"[otcpk, sqzzf, flegg, investor, recorded, enco...",2023-09-19 16:29:00,SQZZF,3.300000,0.0
...,...,...,...,...,...,...,...,...
3938,3995.0,https://seekingalpha.com/article/4621133-banco...,"The Bancorp, Inc. (NASDAQ:TBBK) Q2 2023 Earnin...","[bancorp, tbbk, viroslav, investor, frenkiel, ...",2023-07-28 00:00:00,TBBK,38.500000,747400.0
3939,3996.0,https://seekingalpha.com/article/4621131-natwe...,NatWest Group plc (NYSE:NWG) Q2 2023 Earnings ...,"[natwest, corporate, financial, barclays, deut...",2023-07-28 00:00:00,NWG,6.340000,1444200.0
3940,3997.0,https://seekingalpha.com/article/4621126-cente...,Centene Corporation (NYSE:CNC) Q2 2023 Earning...,"[centene, corporation, investor, financial, ne...",2023-07-28 00:00:00,CNC,66.180000,7128600.0
3941,3998.0,https://seekingalpha.com/article/4621111-costa...,Costamare Inc. (NYSE:CMRE) Q2 2023 Earnings Ca...,"[costamare, cmre, zikos, financial, nokta, cos...",2023-07-28 00:00:00,CMRE,11.590000,4440100.0


,id,url,file_from,tagged,transcript,clean,date,stock,tick,meeting_date,close,volume,sma50_close,sma200_close,sma50_vol,sma200_vol,target
0,0,https://seekingalpha.com/article/4468898-xtant...,0005,"Xtant Medical Holdings, Inc. (NYSE:XTNT) Q3 20...","[Xtant Medical Holdings, Inc. (NYSE:XTNT) Q3 2...",xtant medical xtnt partners financial xtant fi...,2021-11-12,NYSE,XTNT,2021-11-12,1.100000,70500.0,1.154400,1.672250,140486.00,1663013.000,0
1,1,https://seekingalpha.com/article/4468886-uni-s...,0005,Uni-Select Inc. (OTCPK:UNIEF) Q3 2021 Earnings...,[Uni-Select Inc. (OTCPK:UNIEF) Q3 2021 Earning...,otcpk unief legal corporate secretary financia...,2021-11-12,OTCPK,UNIEF,2021-11-12,16.000000,10852.0,14.278522,11.600599,457.12,156.755,1
2,3,https://seekingalpha.com/article/4468882-dents...,0005,Dentsu Group Inc. (OTCPK:DNTUF) Q3 2021 Earnin...,[Dentsu Group Inc. (OTCPK:DNTUF) Q3 2021 Earni...,dentsu otcpk dntuf dentsu network dentsu inter...,2021-11-12,OTCPK,DNTUF,2021-11-12,-28.342836,14267.0,-26.778371,-25.616853,352.42,356.645,0
3,4,https://seekingalpha.com/article/4468879-sgl-c...,0005,SGL Carbon SE (OTCPK:SGLFF) Q3 2021 Earnings C...,[SGL Carbon SE (OTCPK:SGLFF) Q3 2021 Earnings ...,sgl otcpk sglff kellert investor dippold finan...,2021-11-12,OTCPK,SGLFF,2021-11-11,10.830000,400.0,10.796200,9.320700,240.00,318.000,1
4,5,https://seekingalpha.com/article/4468877-konin...,0005,Koninklijke Vopak N.V. (OTCPK:VOPKF) Q3 2021 E...,[Koninklijke Vopak N.V. (OTCPK:VOPKF) Q3 2021 ...,koninklijke vopak otcpk vopkf investor hoekstr...,2021-11-12,OTCPK,VOPKF,2021-11-12,38.812000,0.0,40.800520,45.000936,664.00,695.500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16761,24912,https://seekingalpha.com/article/4445480-arcos...,0200,"Arcosa, Inc. (NYSE:ACA) Q2 2021 Earnings Confe...","[Arcosa, Inc. (NYSE:ACA) Q2 2021 Earnings Conf...",arcosa zaffino thielman sidoti bergner gabelli...,2021-08-05,NYSE,ACA,2021-08-05,51.318615,468700.0,57.036444,58.085233,213020.00,259963.500,0
16762,24913,https://seekingalpha.com/article/4445479-renew...,0200,"Renewable Energy Group, Inc. (REGI) Q2 2021 Ea...","[Renewable Energy Group, Inc. (REGI) Q2 2021 E...",financial treasurer bealmear financial credit ...,2021-08-05,noInfo,REGI,2021-08-05,61.500000,1284541.0,63.793400,69.368850,889098.92,1178279.620,0
16763,24914,https://seekingalpha.com/article/4445472-becto...,0200,Becton Dickinson and Company (NYSE:BDX) Q3 202...,[Becton Dickinson and Company (NYSE:BDX) Q3 20...,investor administrative medical interventional...,2021-08-05,NYSE,BDX,2021-08-05,235.513428,2559118.0,238.028843,237.354383,1341805.20,1391044.180,1
16764,24916,https://seekingalpha.com/article/4445469-bce-i...,0200,BCE Inc. (NYSE:BCE) Q2 2021 Earnings Conferenc...,[BCE Inc. (NYSE:BCE) Q2 2021 Earnings Conferen...,fotopoulos bibic financial securities galappat...,2021-08-05,NYSE,BCE,2021-08-05,48.833820,1588200.0,47.849117,43.219273,1073116.00,1246412.000,1
